# 神經網路計算(二)
## 小批次學習
* 輸入層一次輸入多筆訓練資料，多筆訓練資料整批(同時)由神經網路進行計算 

參考: Deep Learning, 齋藤康毅，OREILLY

In [1]:
import numpy as np

## Activation functions
### Sigmoid函數

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [39]:
X=np.array([[1.0, 0.5], [1.0, 1.5], [0.7, 0.9]])

In [40]:
sigmoid(X)

array([[0.73105858, 0.62245933],
       [0.73105858, 0.81757448],
       [0.66818777, 0.7109495 ]])

### ReLU函數

In [3]:
def relu(x):
    return np.maximum(0, x)

In [41]:
X=np.array([[1.0, 0.5], [-1.0, 1.5], [0.7, -0.9]])

In [42]:
relu(X)

array([[1. , 0.5],
       [0. , 1.5],
       [0.7, 0. ]])

### Softmax函數

In [4]:
def softmax_0(x):
    c=np.max(x)
    x = x - c
    return np.exp(x) / np.sum(np.exp(x))

In [5]:
def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

In [58]:
x=np.array([[1.0, 0.5], [1.0, 1.5], [0.7, 0.9]])

In [59]:
np.max(x)

1.5

In [60]:
np.max(x, axis=-1, keepdims=True)

array([[1. ],
       [1.5],
       [0.9]])

In [61]:
x - np.max(x, axis=-1, keepdims=True)

array([[ 0. , -0.5],
       [-0.5,  0. ],
       [-0.2,  0. ]])

In [62]:
x = x - np.max(x, axis=-1, keepdims=True)
np.exp(x)

array([[1.        , 0.60653066],
       [0.60653066, 1.        ],
       [0.81873075, 1.        ]])

In [63]:
np.sum(np.exp(x), axis=-1, keepdims=True)

array([[1.60653066],
       [1.60653066],
       [1.81873075]])

In [64]:
softmax(x)

array([[0.62245933, 0.37754067],
       [0.37754067, 0.62245933],
       [0.450166  , 0.549834  ]])

In [65]:
softmax_0(x) # 這是錯誤的!

array([[0.19873635, 0.12053969],
       [0.12053969, 0.19873635],
       [0.16271156, 0.19873635]])

## Loss fnctions
### Cross entropy error (交叉熵誤差)

In [66]:
def cross_entropy_error_0(y, t):
    return -np.sum(t * np.log(y + 1e-7))

In [67]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [68]:
y=np.array([[0.9, 0.2], [0.6, 0.4], [0.7, 0.3]])
t=np.array([[1, 0], [0, 1], [1, 0]])

In [69]:
np.log(y + 1e-7)

array([[-0.1053604 , -1.60943741],
       [-0.51082546, -0.91629048],
       [-0.3566748 , -1.20397247]])

In [70]:
t * np.log(y + 1e-7)

array([[-0.1053604 , -0.        ],
       [-0.        , -0.91629048],
       [-0.3566748 , -0.        ]])

In [71]:
np.sum(t * np.log(y + 1e-7))

-1.3783256875025076

In [72]:
cross_entropy_error(y, t)

0.4594418958341692

In [73]:
cross_entropy_error_0(y, t)

1.3783256875025076

## 多層神經網路小批次學習
* 輸入層一次輸入多筆訓練資料，多筆訓練資料整批(同時)由神經網路進行計算

#### 輸入層

In [8]:
X=np.array([[1.0, 0.5], [1.0, 1.5], [0.7, 0.9]])
t=np.array([[1, 0], [0, 1], [1, 0]])

In [92]:
X.shape

(3, 2)

In [9]:
X

array([[1. , 0.5],
       [1. , 1.5],
       [0.7, 0.9]])

#### 第一層

In [10]:
W1=np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
B1=np.array([0.1, 0.2, 0.3])

In [11]:
print(X.shape)
print(W1.shape)
print(B1.shape)

(3, 2)
(2, 3)
(3,)


In [12]:
A1=np.dot(X, W1)+B1
print(A1)

[[0.3  0.7  1.1 ]
 [0.5  1.1  1.7 ]
 [0.35 0.77 1.19]]


In [13]:
Z1=sigmoid(A1)
print(Z1)

[[0.57444252 0.66818777 0.75026011]
 [0.62245933 0.75026011 0.84553473]
 [0.58661758 0.68352089 0.76674106]]


#### 第二層

In [14]:
W2=np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
B2=np.array([0.1, 0.2])

In [15]:
print(Z1.shape)
print(W2.shape)
print(B2.shape)

(3, 3)
(3, 2)
(2,)


In [16]:
A2=np.dot(Z1, W2)+B2
print(A2)

[[0.51615984 1.21402696]
 [0.56595837 1.33143463]
 [0.52538826 1.23645212]]


In [17]:
Z2=relu(A2)
print(Z2)

[[0.51615984 1.21402696]
 [0.56595837 1.33143463]
 [0.52538826 1.23645212]]


#### 第三層(輸出層)

In [18]:
W3=np.array([[0.1, 0.3], [0.2, 0.4]])
B3=np.array([0.1, 0.2])

In [19]:
print(Z2.shape)
print(W3.shape)
print(B3.shape)

(3, 2)
(2, 2)
(2,)


In [20]:
A3=np.dot(Z2, W3)+B3
print(A3)

[[0.39442138 0.84045873]
 [0.42288276 0.90236136]
 [0.39982925 0.85219732]]


In [21]:
y=softmax(A3)
print(y)

[[0.39030333 0.60969667]
 [0.38237525 0.61762475]
 [0.38879788 0.61120212]]


### Cross entropy error (交叉熵誤差)

In [22]:
t=[[1, 0], [0, 1], [1, 0]]

In [23]:
cross_entropy_error(y, t)

0.7891334193966602